In [11]:
import httpx
import os
from dotenv import load_dotenv
import json

load_dotenv('../../.env')

base_url = os.getenv('OGC_API_FEATURES_BASE_URL')

async def get_collection_info(base_url: str, collection_name: str) -> str:
    url = f'{base_url}/collections/{collection_name}.json'
    
    try:
        async with httpx.AsyncClient() as client:
            res = await client.get(url)
            res.raise_for_status()

            data = res.json()
            del data['links']
            return data

    except httpx.HTTPStatusError as e:
        return f"Error response {e.response.status_code}. "

collection_names = ['public.buildings_polygons', 'public.places_points']

res = f"\n\n{'-' * 100}\n\n".join(
    [json.dumps(await get_collection_info(base_url, collection_name), indent=4)
     for collection_name in collection_names]
) + '\n\nTHIS IS FOR YOUR USE ONLY, DO NOT PRESENT ALL THIS INFORMATION TO THE HUMAN.'

print(res)

{
    "id": "public.buildings_polygons",
    "title": "public.buildings_polygons",
    "description": "Data for table public.buildings_polygons",
    "extent": {
        "spatial": {
            "crs": "http://www.opengis.net/def/crs/EPSG/0/4326",
            "bbox": [
                -8.732908248901367,
                57.967613220214844,
                31.156585693359375,
                80.69022369384766
            ]
        }
    },
    "geometrytype": "MultiPolygon",
    "properties": [
        {
            "name": "id",
            "type": "number",
            "description": ""
        },
        {
            "name": "osm_id",
            "type": "string",
            "description": ""
        },
        {
            "name": "code",
            "type": "number",
            "description": ""
        },
        {
            "name": "fclass",
            "type": "string",
            "description": ""
        },
        {
            "name": "name",
            "type": "stri

In [22]:
url = f'{base_url}/collections.json'

async with httpx.AsyncClient() as client:
    res = await client.get(url)
    res.raise_for_status()

    data = res.json()
    ya = '\n'.join(
        [f"`{collection['title']}` -- {collection['description']}"
         for collection in data['collections']]
    )
    print(ya)

`public.buildings_polygons` -- Data for table public.buildings_polygons
`public.landuse_polygons` -- Data for table public.landuse_polygons
`public.natural_points` -- Data for table public.natural_points
`public.natural_polygons` -- Data for table public.natural_polygons
`public.places_of_worship_points` -- Data for table public.places_of_worship_points
`public.places_of_worship_polygons` -- Data for table public.places_of_worship_polygons
`public.places_points` -- Data for table public.places_points
`public.places_polygons` -- Data for table public.places_polygons
`public.points_of_interest_points` -- Data for table public.points_of_interest_points
`public.points_of_interest_polygons` -- Data for table public.points_of_interest_polygons
`public.railways_lines` -- Data for table public.railways_lines
`public.roads_lines` -- Data for table public.roads_lines
`public.traffic_points` -- Data for table public.traffic_points
`public.traffic_polygons` -- Data for table public.traffic_polygon

In [21]:
import re

s = "public.something"
s = re.sub(r'^public\.', '', 'public.ye')
print(s)

ye


In [ ]:
import geopandas as gpd

# Read the polygon GeoJSON file
nidarosdomen = gpd.read_file('/tmp/tmp5_s4jbwg/Nidarosdomen_polygon.geojson')

# Create a 20 meter buffer around the polygon
buffered_polygon = nidarosdomen.buffer(20)

# Convert the buffered polygon to a GeoDataFrame
buffered_gdf = gpd.GeoDataFrame(geometry=buffered_polygon)
buffered_gdf

In [30]:
url = f'{base_url}/collections/public.osm_buildings_polygons/items.json?limit=5'

async with httpx.AsyncClient() as client:
    res = await client.get(url)
    res.raise_for_status()

    data = res.json()
    for f in data['features']:
        f['geometry'] = '[...]'
    print(json.dumps(data, indent=4))

{
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "id": "101073",
            "geometry": "[...]",
            "properties": {
                "code": 1500,
                "fclass": "building",
                "id": 101073,
                "name": null,
                "osm_id": "132121353",
                "type": "house"
            }
        },
        {
            "type": "Feature",
            "id": "101074",
            "geometry": "[...]",
            "properties": {
                "code": 1500,
                "fclass": "building",
                "id": 101074,
                "name": null,
                "osm_id": "132121354",
                "type": "house"
            }
        },
        {
            "type": "Feature",
            "id": "101075",
            "geometry": "[...]",
            "properties": {
                "code": 1500,
                "fclass": "building",
                "id": 101075,
           

In [74]:
from collections import Counter

url = f'{base_url}/collections/public.osm_places_polygons/items.json?limit=5000'

async with httpx.AsyncClient() as client:
    res = await client.get(url, timeout=None)
    res.raise_for_status()

    data = res.json()

    property_counters = {}

    for feature in data['features']:
        for key, value in feature['properties'].items():
            if key in ['id', 'code', 'osm_id', 'name', 'ref', 'layer']:
                continue
            
            if key not in property_counters:
                property_counters[key] = Counter()

            property_counters[key][value] += 1

total_counts = {key: sum(counter.values()) for key, counter in property_counters.items()}

percentages = {}
cutoff = 0.0
for key, counter in property_counters.items():
    percentages[key] = {}
    other_count = 0
    for value, count in counter.items():
        percentage = count / total_counts[key] * 100
        if percentage >= cutoff:
            percentages[key][value] = percentage
        else:
            other_count += count
    if other_count > 0:
        percentages[key]['other'] = other_count / total_counts[key] * 100

print("Percentages for each property (values below 1% grouped into 'other'):")
for key, values in percentages.items():
    print(f"Property: {key}")
    sorted_values = sorted(values.items(), key=lambda x: x[1], reverse=True)
    for value, percentage in sorted_values:
        print(f"    {value}: {percentage:.1f}%")

Percentages for each property (values below 1% grouped into 'other'):
Property: fclass
    island: 78.9%
    farm: 16.5%
    locality: 3.3%
    suburb: 0.7%
    hamlet: 0.3%
    village: 0.1%
    county: 0.1%
    town: 0.0%
    city: 0.0%
Property: population
    0: 99.9%
    11173: 0.0%
    13358: 0.0%
    4: 0.0%
    20: 0.0%
    241084: 0.0%
    492350: 0.0%


In [80]:
a = {'a': 1} 
b = {'a': 2}

a.update(b)

a

{'a': 2}